In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Import the data
df = pd.read_excel('innercity.xlsx')

In [3]:
# Ignore the rows which has $ as value in any column
df = df[~df.isin(['$']).any(axis=1)]

In [4]:
df.head()

,cid,dayhours,price,room_bed,room_bath,living_measure,lot_measure,ceil,coast,sight,...,basement,yr_built,yr_renovated,zipcode,lat,long,living_measure15,lot_measure15,furnished,total_area
0,3876100940,20150427T000000,600000,4.0,1.75,3050.0,9440.0,1,0,0.0,...,1250.0,1966,0,98034,47.7228,-122.183,2020.0,8660.0,0.0,12490
1,3145600250,20150317T000000,190000,2.0,1.00,670.0,3101.0,1,0,0.0,...,0.0,1948,0,98118,47.5546,-122.274,1660.0,4100.0,0.0,3771
2,7129303070,20140820T000000,735000,4.0,2.75,3040.0,2415.0,2,1,4.0,...,0.0,1966,0,98118,47.5188,-122.256,2620.0,2433.0,0.0,5455
3,7338220280,20141010T000000,257000,3.0,2.50,1740.0,3721.0,2,0,0.0,...,0.0,2009,0,98002,47.3363,-122.213,2030.0,3794.0,0.0,5461
4,7950300670,20150218T000000,450000,2.0,1.00,1120.0,4590.0,1,0,0.0,...,0.0,1924,0,98118,47.5663,-122.285,1120.0,5100.0,0.0,5710


In [5]:
# outlier treatment
df = df[df['room_bed'] < 8]
df = df[df['room_bath'] < 5]
df = df[df['living_measure'] < 6000]
df = df[df['lot_measure'] < 100000]
df = df[df['ceil'] < 4]
df = df[df['coast'] < 2]
df = df[df['sight'] < 5]
df = df[df['condition'] < 5]
df = df[df['quality'] < 12]
df = df[df['ceil_measure'] < 6000]
df = df[df['basement'] < 4000]
df = df[df['yr_built'] > 1900]
df = df[df['yr_renovated'] < 2015]
df = df[df['zipcode'] < 98080]
df = df[df['lat'] > 47]
df = df[df['long'] < -120]
df = df[df['living_measure15'] < 6000]
df = df[df['lot_measure15'] < 100000]
df = df[df['total_area'] < 100000]

In [6]:
df.head()

,cid,dayhours,price,room_bed,room_bath,living_measure,lot_measure,ceil,coast,sight,...,basement,yr_built,yr_renovated,zipcode,lat,long,living_measure15,lot_measure15,furnished,total_area
0,3876100940,20150427T000000,600000,4.0,1.75,3050.0,9440.0,1,0,0.0,...,1250.0,1966,0,98034,47.7228,-122.183,2020.0,8660.0,0.0,12490
3,7338220280,20141010T000000,257000,3.0,2.50,1740.0,3721.0,2,0,0.0,...,0.0,2009,0,98002,47.3363,-122.213,2030.0,3794.0,0.0,5461
7,1624059224,20140618T000000,1160000,4.0,3.50,4680.0,9700.0,2,0,0.0,...,1320.0,2005,0,98006,47.5703,-122.165,2800.0,12343.0,1.0,14380
8,5469503280,20140721T000000,449950,4.0,2.50,3100.0,10000.0,2,0,0.0,...,0.0,1978,0,98042,47.3741,-122.15,1850.0,9438.0,1.0,13100
9,2787460430,20141028T000000,299950,2.0,1.75,1460.0,10506.0,1,0,0.0,...,0.0,1983,0,98031,47.4048,-122.178,1460.0,8153.0,0.0,11966


In [7]:
# drop the cid column
df = df.drop(['cid'], axis=1)
df = df.drop(['dayhours'], axis=1)
# Ignore redundant columns
df = df.drop(['basement'], axis=1)
df = df.drop(['yr_renovated'], axis=1)


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
var = ['living_measure','lot_measure','ceil_measure','living_measure15','lot_measure15','total_area']
df[var] = scaler.fit_transform(df[var])


In [9]:
# Encode using one hot encoding
df = pd.get_dummies(df, columns=['room_bed','room_bath','ceil','coast','sight','condition','quality','zipcode'])

C:\Users\hp\AppData\Local\Temp\ipykernel_22264\1098211735.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  df = pd.get_dummies(df, columns=['room_bed','room_bath','ceil','coast','sight','condition','quality','zipcode'])


In [10]:
# split the data into train and test
from sklearn.model_selection import train_test_split
X = df.drop(['price'], axis=1)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=100)

In [11]:
# Load Linear Regression model
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

In [12]:
lm.fit(X_train, y_train)

LinearRegression()

In [13]:
# predict the values
y_pred = lm.predict(X_test)

In [14]:
# evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
print('Mean squared error: %.2f' % mean_squared_error(y_test, y_pred))
print('Coefficient of determination: %.2f' % r2_score(y_test, y_pred))


Mean squared error: 15625649747.69
Coefficient of determination: 0.85
